# regression homework in week 9
**info**

苏锦华 2017201620

**requiresment**

do outlier test for the question 11 in the chapter 3, for each line in the table the following results should be output. Results will be drawn by metrix manipulation rather than any packages.

| symbol | interpretation |
| :-- | :--: |
| $e_{i}$ | residual error |
| $SRE_{i}$ | student's residual error |
| $e_{(i)}$ | residual error without i-th line |
| $SRE_{(i)}$ | student's residual error without i-th line |
| $ch_{i}$ | centeralized leverage value |
| $D_{i}$ | cook distance |


In [1]:
import pandas as pd
df = pd.read_csv('regression_w9_data.csv')
df

,编号,货运总量y(万吨),工业总产值x1(亿元),农业总产值x3(亿元),居民非商品支出x3(亿元)
0,1,160,70,35,1.0
1,2,260,75,40,2.4
2,3,210,65,40,2.0
3,4,265,74,42,3.0
4,5,240,72,38,1.2
5,6,220,68,45,1.5
6,7,275,78,42,4.0
7,8,160,66,36,2.0
8,9,275,70,44,3.2
9,10,250,65,42,3.0


In [92]:
class outlier_test(object):
    def __init__(self, y, X):
        self.y = y
        self.X = np.hstack((np.ones((10,1)),X))
        self.test()
    def test(self):
        self.n_minus_p = len(self.X) - len(self.X[0]) + 1
        self.coef = np.linalg.inv(self.X.T.dot(self.X)).dot(self.X.T.dot(y))
        self.y_pred = self.X.dot(self.coef)
        self.error = self.y - self.y_pred
        self.sigma2_estimate = np.sum(self.error ** 2) / (self.n_minus_p - 1)
        self.h = self.X.dot(np.linalg.inv(self.X.T.dot(self.X))).dot(self.X.T).diagonal()
        self.ch = self.h - (1/len(self.X))
        self.SRE = self.error / np.sqrt(self.sigma2_estimate*(1-self.h))
        self.e_delete = self.error /(1-self.h)
        self.SRE_delete = self.SRE * np.sqrt((self.n_minus_p - 2)/(self.n_minus_p - 1 - self.SRE**2))
        self.D = self.error ** 2 * self.h / (1-self.h) ** 2 / (self.sigma2_estimate * (len(self.X[0])))

In [93]:
import numpy as np
X = np.array(df.iloc[:,2:])
y = np.array(df.iloc[:,1])

In [94]:
ot = outlier_test(y,X)
ot.coef

array([-348.28016949,    3.75403675,    7.1007124 ,   12.44746988])

regression result: $y = -348 + 3.754x_1 + 7.1x_2 + 12.447x_3$

In [99]:
res_dict = dict(zip(['x_'+str(i+1) for i in range(len(X.T))],X.T))
res_dict.update({
    'y':y,
    'e_i':ot.error,
    'SRE_i':ot.SRE,
    'e_(i)':ot.e_delete, 
    'SRE_(i)':ot.SRE_delete,
    'h(i)':ot.h,
    'ch_i':ot.ch,
    'D_i':ot.D,
})
res_df = pd.DataFrame(res_dict)
res_df

,x_1,x_2,x_3,y,e_i,SRE_i,e_(i),SRE_(i),h(i),ch_i,D_i
0,70.0,35.0,1.0,160,-15.474807,-0.893528,-28.351503,-0.876041,0.454180,0.354180,0.166087
1,75.0,40.0,2.4,260,12.824989,0.627667,16.880518,0.592770,0.240249,0.140249,0.031145
2,65.0,40.0,2.0,210,5.344345,0.265166,7.229790,0.243493,0.260788,0.160788,0.006201
3,74.0,42.0,3.0,265,-0.090881,-0.004333,-0.113508,-0.003955,0.199347,0.099347,0.000001
4,72.0,38.0,1.2,240,33.225488,1.753995,50.882734,2.293834,0.347018,0.247018,0.408741
5,68.0,45.0,1.5,220,-25.197593,-2.115660,-97.615228,-3.832135,0.741868,0.641868,3.216007
6,78.0,42.0,4.0,275,-17.554498,-1.173475,-43.106646,-1.220391,0.592766,0.492766,0.501103
7,66.0,36.0,2.0,160,-20.006842,-1.162813,-37.138676,-1.206058,0.461294,0.361294,0.289458
8,70.0,44.0,3.2,275,8.234348,0.409354,11.182868,0.379017,0.263664,0.163664,0.015001
9,65.0,42.0,3.0,250,18.695450,1.064620,33.314858,1.079114,0.438825,0.338825,0.221576


In [100]:
res_df.describe()

,x_1,x_2,x_3,y,e_i,SRE_i,e_(i),SRE_(i),h(i),ch_i,D_i
count,10.00000,10.000000,10.000000,10.000000,1.000000e+01,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000
mean,70.30000,40.400000,2.330000,231.500000,-9.310952e-12,-0.122901,-8.683479,-0.255035,0.400000,0.300000,0.485532
std,4.44847,3.272783,0.966149,43.400589,1.914022e+01,1.187637,43.432195,1.657532,0.172816,0.172816,0.975543
min,65.00000,35.000000,1.000000,160.000000,-2.519759e+01,-2.115660,-97.615228,-3.832135,0.199347,0.099347,0.000001
25%,66.50000,38.500000,1.625000,212.500000,-1.703457e+01,-1.095492,-34.941883,-1.123554,0.261507,0.161507,0.019037
50%,70.00000,41.000000,2.200000,245.000000,2.626732e+00,0.130417,3.558141,0.119769,0.392922,0.292922,0.193832
75%,73.50000,42.000000,3.000000,263.750000,1.167733e+01,0.573088,15.456106,0.539332,0.459515,0.359515,0.378920
max,78.00000,45.000000,4.000000,275.000000,3.322549e+01,1.753995,50.882734,2.293834,0.741868,0.641868,3.216007


**conclusion**
- The mean of ch is 0.3, so the 5-th data may be seen as x outlier since the ch of 5-th is 0.6418.
- The SRE_(5) is -3.832, so the 5-th data is significantly different from other y.
- Indeed the 5-th data also has biggest cook distance value.